In [ ]:
%%capture
import scikits.samplerate
import sklearn
import os
import site
import numpy
import matplotlib

#"""

from mpl_toolkits.mplot3d import proj3d
def orthogonal_proj(zfront, zback):
    a = (zfront+zback)/(zfront-zback)
    b = -2*(zfront*zback)/(zfront-zback)
    return numpy.array([[1,0,0,0],
                        [0,1,0,0],
                        [0,0,a,b],
                        [0,0,-0.00001,1]])
proj3d.persp_transformation = orthogonal_proj

#"""

site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2
import logging
import theano.tensor as T
import lasagne
import logging
log = logging.getLogger()
log.setLevel("DEBUG")
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

import matplotlib.lines as mlines
#import seaborn
#seaborn.set_style('whitegrid')
matplotlib.rcParams['figure.figsize'] = (16.0, 1.0)
matplotlib.rcParams['font.size'] = 7
from braindevel.scripts.print_results import ResultPrinter
import theano
import scipy.signal
import numpy as np


In [ ]:
%%javascript
function makeSVGDownloadable() {
    $('body').on('click', 'svg', function(){
        var svg = this.outerHTML;
        var b64 = btoa(svg); // or use btoa if supported
        var linkToSVG = $("<a href-lang='image/svg+xml' href='data:image/svg+xml;base64,\n" + b64 +
                          "'title='file.svg' target='blank'>Download</a>");
        $(this).after(linkToSVG);});
}
makeSVGDownloadable()

In [ ]:
import seaborn

seaborn.set_style('white')

In [ ]:
# see also http://openscholarship.wustl.edu/cgi/viewcontent.cgi?article=1150&context=eng_etds#page=14
from matplotlib.collections import PolyCollection
def cuboid_data(start_point, size):
    """
       Create a data array for cuboid plotting.


       ============= ================================================
       Argument      Description
       ============= ================================================
       start_point   Lower bottom left part of cuboid
       size          size of the cuboid, triple, (x_length,y_width,z_height)
       :type size: tuple, numpy.array, list
       :param size: size of the cuboid, triple, (x_length,y_width,z_height)
       :type center: tuple, numpy.array, list
       :param center: center of the cuboid, triple, (x,y,z)


      """


    # suppose axis direction: x: to left; y: to inside; z: to upper
    # get the (left, outside, bottom) point
    o = start_point
    # get the length, width, and height
    l, w, h = size
    x = [[o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in bottom surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in upper surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in outside surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]]]  # x coordinate of points in inside surface
    y = [[o[1], o[1], o[1] + w, o[1] + w, o[1]],  # y coordinate of points in bottom surface
         [o[1], o[1], o[1] + w, o[1] + w, o[1]],  # y coordinate of points in upper surface
         [o[1], o[1], o[1], o[1], o[1]],          # y coordinate of points in outside surface
         [o[1] + w, o[1] + w, o[1] + w, o[1] + w, o[1] + w]]    # y coordinate of points in inside surface
    z = [[o[2], o[2], o[2], o[2], o[2]],                        # z coordinate of points in bottom surface
         [o[2] + h, o[2] + h, o[2] + h, o[2] + h, o[2] + h],    # z coordinate of points in upper surface
         [o[2], o[2], o[2] + h, o[2] + h, o[2]],                # z coordinate of points in outside surface
         [o[2], o[2], o[2] + h, o[2] + h, o[2]]]                # z coordinate of points in inside surface
    return x, y, z

def orthogonal_proj(zfront, zback):
    a = (zfront+zback)/(zfront-zback)
    b = -2*(zfront*zback)/(zfront-zback)
    return np.array([[1,0,0,0],
                        [-0.86602686,1,0,0],
                        [0,0,a,b],
                        [0,0,-1e-5,zback]])

proj3d.persp_transformation = orthogonal_proj


def add_rect(ax, xz, length, height, ys=0,**polyargs):
    poly = PolyCollection(np.array([[[xz[0],xz[1]],
                                     [xz[0]+length,xz[1]],
                                     [xz[0]+length,xz[1]+height],
                                     [xz[0],xz[1]+height]]]),
                         **polyargs)
    ax.add_collection3d(poly, zs=ys, zdir='y')
    return poly

def plot_featuremap(xyz, length, width, height, l_str=None, w_str=None, h_str=None, **annotate_args):
    return plot_cuboid(xyz, length, width, height, featuremap_color,
                      l_str=l_str, w_str=w_str, h_str=h_str, text_color='black',
                      **annotate_args)
    
def plot_filters(xyz, length, width, height, l_str=None, w_str=None, h_str=None, topleft_annotate=True,
                 **annotate_args):
    return plot_cuboid(xyz, length, width, height, filter_color,
                        l_str=l_str, w_str=w_str, h_str=h_str, text_color=text_filter_color,
                       topleft_annotate=topleft_annotate,
                      **annotate_args)
    

def plot_cuboid(xyz, length, width, height, color, l_str=None, w_str=None, h_str=None,
                text_color='black', topleft_annotate=False, **annotate_args):
    X, Y, Z = cuboid_data(xyz, (length, width, height))
    cuboid = ax.plot_surface(X, Y, Z, color=color,
                    rstride=1, cstride=1, shade=False,
                    edgecolors='black')
    if l_str is not None:
        assert (w_str is not None) and (h_str is not None)
        if topleft_annotate: 
            annotate_cub_sizes_topleft(xyz,length,width,height,
                           l_str, w_str, h_str,
                           color=text_color,
                           **annotate_args)
        else:
             annotate_cub_sizes(xyz,length,width,height,
                           l_str, w_str, h_str,
                           color=text_color,
                           **annotate_args)
    return cuboid

def annotate_cub_sizes(xyz, length, width, height, 
                       xtext, ytext, ztext,fontsize=12, boxpad=-0.5,boxalpha=1,**text_args):
    ax.text3D(*(np.array(xyz) + [length / 16.0, 0, 0]), s=xtext, 
          transform=offset_copy(ax.transData, fig, x=0.0, y=-0.05),
              va='top', fontsize=fontsize, backgroundcolor='white',
              bbox=dict(pad=boxpad, color='white', alpha=boxalpha), zorder= 10,
              **text_args)
    ax.text3D(*(np.array(xyz) + [0, width / 2.0,0]),s=ytext, 
              transform=offset_copy(ax.transData,fig,x=-0.05,y=-0.05),
             ha='right', va='center', fontsize=fontsize, backgroundcolor='white',
              bbox=dict(pad=boxpad, color='white', alpha=boxalpha), zorder= 10,
              **text_args)
    ax.text3D(*(np.array(xyz) + [0, width, height/2.0]), s=ztext, 
              transform=offset_copy(ax.transData, fig, x=-0.05,y=0),
              ha='right', va='center', fontsize=fontsize, backgroundcolor='white',
              bbox=dict(pad=boxpad, color='white', alpha=boxalpha), zorder= 10,
              **text_args)
    
def annotate_cub_sizes_topleft(xyz, length, width, height, 
                       xtext, ytext, ztext,fontsize=12, boxpad=-0.5,boxalpha=1,**text_args):
    top_left_xyz = np.array(xyz) + np.array([length,width,height])
    ax.text3D(*(top_left_xyz - [length/2.0, 0, 0]), s=xtext, 
          transform=offset_copy(ax.transData, fig, x=0.0, y=+0.025),
              ha='center',
              va='bottom', fontsize=fontsize, backgroundcolor='white',
              bbox=dict(pad=boxpad, color='white', alpha=boxalpha), zorder= 10,
              **text_args)
    ax.text3D(*(top_left_xyz - [0, width / 2.0,0]),s=ytext, 
              transform=offset_copy(ax.transData,fig,x=+0.05,y=+0.0),
             ha='left', va='center', fontsize=fontsize, backgroundcolor='white',
              bbox=dict(pad=boxpad, color='white', alpha=boxalpha), zorder= 10,
              **text_args)
    ax.text3D(*(top_left_xyz - [0, width, height/2.0]), s=ztext, 
              transform=offset_copy(ax.transData, fig, x=+0.05,y=0),
              ha='left', va='center', fontsize=fontsize, backgroundcolor='white',
              bbox=dict(pad=boxpad, color='white', alpha=boxalpha), zorder= 10,
              **text_args)

def draw_vertical_connector(z_start,z_end):
    ax.plot3D([0,0], [0,0], zs=[z_start,z_end], linestyle='dotted',
         linewidth=1., color='black')

In [ ]:
from braindevel.datasets.loaders import BBCIDataset
from braindevel.mywyrm.processing import resample_cnt, exponential_standardize_cnt
from matplotlib.transforms import offset_copy

In [ ]:
cnt = BBCIDataset('data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat', load_sensor_names=['C3', 'C4',
                                                                                                  'CP4']).load()

cnt = resample_cnt(cnt, 250,)
cnt = exponential_standardize_cnt(cnt)

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#%config InlineBackend.figure_format = 'svg' 
#fig = plt.figure(figsize=(14,26))


%config InlineBackend.figure_format = 'svg' 
fig = plt.figure(figsize=(10,20))
ax = fig.gca(projection='3d')

ax.set_xlim3d(-100, 900)
ax.set_ylim3d(-100, 800)
ax.set_zlim3d(-1000, 0)
ax.view_init(30,210)

#plt.tight_layout()
plt.axis('off')
no_color = (1,1,1,0)
black = (0,0,0,1)

featuremap_color = (1,1,1,0.0)
brown = seaborn.color_palette("BrBG")[0]#(132/255.0, 98/255.0, 98/255.0)
filter_color = seaborn.color_palette()[4] + (0.5,)#brown + (0.5,) #seaborn.color_palette()[4] + (0.7,)
activation_color = brown
text_filter_color = brown

# rough width to height conversion
width_factor = 1.3 / 2.0

one_height = 0
one_width = 0
length_f0, width_f0, height_f0 = 1000, one_width, 90
feature_0_xyz = -length_f0/2.0, -width_f0/2.0, 0
feature_0 = plot_featuremap(feature_0_xyz, length_f0, width_f0, height_f0,
                           "534", "", "44")

for i_chan, z_offset in enumerate((20,45,70)):
    ax.plot3D(np.linspace(feature_0_xyz[0],
                          feature_0_xyz[0]+length_f0,200), 
                          [feature_0_xyz[1]] * 200,
                          zs=9/2.0*cnt.data[12035:12235, i_chan] + z_offset,
                         color=seaborn.color_palette()[0])

    
filters_0_xyz = (-75, feature_0_xyz[1], 2)
filters_0 = plot_filters(filters_0_xyz, 150, width_f0, 28,
                         "25", "", "1", boxpad=1, boxalpha=1)

# first featuremap
length_f1, width_f1, height_f1 = 975, 40 * 0.6, 90
feature_1_xyz = -length_f1/2.0, -width_f1/2.0, -170
feature_1 = plot_featuremap(feature_1_xyz, length_f1,width_f1, height_f1, '510', '40', ' 44')
filters_1_xyz = (feature_1_xyz[0] + (length_f1)/2 -10,
              feature_1_xyz[1], feature_1_xyz[2]) 
filters_1 = plot_filters(filters_1_xyz, 0, width_f1, height_f1, '','40','44')

# square
square_z = feature_1_xyz[2] - 75
ax.text3D(0,0,square_z, 'Square', ha='center', fontsize=12, weight='bold',
         backgroundcolor='white')

#second featuremap
length_f2, width_f2, height_f2 = 975, 40 * 0.6, one_height
feature_2_xyz = -length_f2/2.0, -width_f2/2.0, square_z-75
plot_featuremap(feature_2_xyz, length_f2,width_f2, height_f2,
                  '510', '40', '')
pool_len = 200
filter_2_xyz = (-pool_len / 2.0, 0, feature_2_xyz[2]+2)
plot_filters(filter_2_xyz, pool_len, 7, height_f2,
                  '75', '1', '',)
# log
log_z = feature_2_xyz[2] - 65
ax.text3D(0,0,log_z, 'Log', ha='center', fontsize=12, weight='bold',
         backgroundcolor='white')

# final softmax
length_soft, width_soft, height_soft = 100 * 2, 20 * 2 * 0.6, one_height
feature_soft_xyz = -length_soft/2.0, -width_soft/2.0, log_z-65
plot_filters(feature_soft_xyz,
            length_soft, width_soft, height_soft, '30', '40', '')

# final outs 

text_z = feature_soft_xyz[2] - 110

text_dist = 400

ax.text3D(-text_dist - text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Hand (L)', fontsize=12,va='top', ha='center')
ax.text3D(-text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Hand (R)', fontsize=12,va='top', ha='center')
ax.text3D(text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Feet', fontsize=12,va='top', ha='center')
ax.text3D(text_dist + text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Rest', fontsize=12,va='top', ha='center')

# connectors to classes
ax.plot3D([0, -text_dist - text_dist / 2.0], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([0,-text_dist / 2.0,], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([0,text_dist / 2.0], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([0,text_dist + text_dist / 2.0], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')


explain_fontsize=12
text_x_headers = -1300 
ax.text3D(text_x_headers, 0, feature_0_xyz[2] + height_f0 * 1 + width_f0 * width_factor,
          'Convolution\n(temporal)', 
           weight='bold', ha='center',
         fontsize=explain_fontsize,  va='top',)
ax.text3D(text_x_headers, 0, feature_1_xyz[2] + height_f1 * 1 + width_f1 * width_factor,
          'Spatial filter\n(all electrodes,\nall previous filters)', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, feature_2_xyz[2] + height_f2 * 1 + width_f2 * width_factor,
          'Mean Pooling\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, feature_soft_xyz[2] + height_soft * 1 + width_soft * width_factor,
          'Linear Classification\n(Dense Layer+Softmax)', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')

text_x_filt = text_x_headers 
offset_z = -52
ax.text3D(text_x_filt, 0, feature_0_xyz[2] + height_f0 * 1 + width_f0 * width_factor+offset_z,'40 Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, feature_1_xyz[2] + height_f1 * 1 + width_f1 * width_factor+offset_z-23,'40 Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, feature_2_xyz[2] + height_f2 * 1 + width_f2 * width_factor+offset_z/2.0, 
          'Stride 15x1', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, feature_soft_xyz[2] + height_soft * 1 + width_soft * width_factor+offset_z, '4 Units', ha='center',
         fontsize=explain_fontsize,  va='top')

ax.set_zlim(-1800,-800)
plt.savefig('3D_Diagram_MatplotLib.ipynb.0.pdf', bbox_inches='tight', dpi=600)

### horizontal plots

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#%config InlineBackend.figure_format = 'svg' 
#fig = plt.figure(figsize=(14,26))


%config InlineBackend.figure_format = 'svg' 
fig = plt.figure(figsize=(10,20))
ax = fig.gca(projection='3d')

ax.set_xlim3d(1500, 2500)
ax.set_ylim3d(-100, 800)
ax.set_zlim3d(-1800, -800)
ax.view_init(30,210)

#plt.tight_layout()
plt.axis('off')
no_color = (1,1,1,0)
black = (0,0,0,1)

featuremap_color = (1,1,1,0.0)
brown = seaborn.color_palette("BrBG")[0]#(132/255.0, 98/255.0, 98/255.0)
filter_color = seaborn.color_palette()[4] + (0.5,)#brown + (0.5,) #seaborn.color_palette()[4] + (0.7,)
activation_color = brown
text_filter_color = brown

# rough width to height conversion
width_factor = 1.3 / 2.0

one_height = 0
one_width = 0
length_f0, width_f0, height_f0 = 450, one_width, 120
feature_0_xyz = -480, -width_f0/2.0, -height_f0 / 2.0
feature_0 = plot_featuremap(feature_0_xyz, length_f0, width_f0, height_f0,
                           "609", "", "21")

start =20*height_f0/90.0
step = 25*height_f0/90.0
stop = start + step * 2 + 1
z_offsets = np.arange(start,stop,step) + feature_0_xyz[2]
for i_chan, z_offset in enumerate(z_offsets):
    ax.plot3D(np.linspace(feature_0_xyz[0],
                          feature_0_xyz[0]+length_f0,200), 
                          [feature_0_xyz[1]] * 200,
                          zs=9/2.0*cnt.data[12035:12235, i_chan] + z_offset,
                         color=seaborn.color_palette()[0])

filter_len = 150
filters_0_xyz = (feature_0_xyz[0] + length_f0/2.0 - filter_len/2.0, feature_0_xyz[1],
                 5*height_f0/90.0 + feature_0_xyz[2])
filters_0 = plot_filters(filters_0_xyz, 150, width_f0, 40,
                         "25", "", "1", boxpad=1, boxalpha=1)

# first featuremap
length_f1, width_f1, height_f1 = 420, 40 * 0.6, 90
feature_1_xyz = feature_0_xyz[0] + length_f0 + 300, -width_f1/2.0, -height_f1 /2.0
feature_1 = plot_featuremap(feature_1_xyz, length_f1,width_f1, height_f1, '585', '40', ' 21')
filters_1_xyz = (feature_1_xyz[0] + (length_f1)/2 -10,
              feature_1_xyz[1], feature_1_xyz[2]) 
filters_1 = plot_filters(filters_1_xyz, 0, width_f1, height_f1, '','40','21')



# square
square_x = feature_1_xyz[0] + length_f1 + 150
ax.text3D(square_x,0,0, 'Square', ha='left', fontsize=12, weight='bold',
         backgroundcolor='white')

#second featuremap
length_f2, width_f2, height_f2 = 420, 40 * 0.6, one_height
feature_2_xyz = square_x + 500, -width_f2/2.0, -height_f2/2.0
plot_featuremap(feature_2_xyz, length_f2,width_f2, height_f2,
                  '585', '40', '')
pool_len = 200
filter_2_xyz = (feature_2_xyz[0] + (pool_len)/2 , feature_2_xyz[1], width_f2 * 0.4)
plot_filters(filter_2_xyz, pool_len, 7, height_f2,
                  '75', '1', '',)
# log
log_x = feature_2_xyz[0] + length_f2 + 150
ax.text3D(log_x,0,0, 'Log', ha='left', fontsize=12, weight='bold',
         backgroundcolor='white')
# final softmax
length_soft, width_soft, height_soft = 100 * 2, 20 * 2 * 0.6, one_height
feature_soft_xyz = log_x + 350, -width_soft/2.0, -height_soft / 2.0
plot_filters(feature_soft_xyz,
            length_soft, width_soft, height_soft, '35', '40', '',topleft_annotate=False)

# final outs 
end_soft_x = feature_soft_xyz[0] + length_soft - width_soft * 0.8 # geraten :D
text_x = end_soft_x + 300

text_dist = 50

ax.text3D(text_x, 0, 0.5 * text_dist, zdir=(0,0,0),s='Normal', fontsize=10,va='center', ha='left')
ax.text3D(text_x, 0, -0.5 * text_dist, zdir=(0,0,0),s='Pathological', fontsize=10,va='center', ha='left')

# connector

middle_soft_z = feature_2_xyz[2] + width_soft * 0.6
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, 0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, -0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')



explain_fontsize=11
text_z_headers = 200 
ax.text3D(feature_0_xyz[0], 0, text_z_headers,
          'Convolution\n(temporal)', 
           weight='bold', ha='left',
         fontsize=explain_fontsize,  va='top',)
ax.text3D(feature_1_xyz[0] - 23, 0, text_z_headers,
          'Spatial filter\n(all electrodes,\nall previous filters)', 
          ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(feature_2_xyz[0] - 23, 0, text_z_headers,
          'Mean Pooling\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(feature_soft_xyz[0] - 23, 0, text_z_headers,
          'Linear Classification\n(Dense Layer+Softmax)', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')

text_z_filt = text_z_headers - 75 
ax.text3D(feature_0_xyz[0], 0, text_z_filt,'40 Units', ha='left',
         fontsize=explain_fontsize,  va='top')
ax.text3D(feature_1_xyz[0] - 23, 0, text_z_filt,'40 Units', ha='left',
         fontsize=explain_fontsize,  va='top')
ax.text3D(feature_2_xyz[0] - 23, 0, text_z_filt, 
          'Stride 15x1', ha='left',
         fontsize=explain_fontsize,  va='top')
ax.text3D(feature_soft_xyz[0] - 23, 0, text_z_filt, '2 Units', ha='left',
         fontsize=explain_fontsize,  va='top')

ax.set_zlim(-1800,-800)
plt.savefig('/home/schirrmr/code/auto-diagnosis/ShallowNet.svg', bbox_inches='tight', dpi=600, pad_inches=0)
plt.savefig('/home/schirrmr/code/auto-diagnosis/ShallowNet.pdf', bbox_inches='tight', dpi=600, pad_inches=0)

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#%config InlineBackend.figure_format = 'svg' 
#fig = plt.figure(figsize=(14,26))


%config InlineBackend.figure_format = 'svg' 
fig = plt.figure(figsize=(10,20))
ax = fig.gca(projection='3d')

ax.set_xlim3d(1500, 2500)
ax.set_ylim3d(-100, 800)
ax.set_zlim3d(-1800, -800)
ax.view_init(30,210)

#plt.tight_layout()
plt.axis('off')
no_color = (1,1,1,0)
black = (0,0,0,1)

featuremap_color = (1,1,1,0.0)
brown = seaborn.color_palette("BrBG")[0]#(132/255.0, 98/255.0, 98/255.0)
filter_color = seaborn.color_palette()[4] + (0.5,)#brown + (0.5,) #seaborn.color_palette()[4] + (0.7,)
activation_color = brown
text_filter_color = brown

# rough width to height conversion
width_factor = 1.3 / 2.0

one_height = 0
one_width = 0
length_f0, width_f0, height_f0 = 450, one_width, 120
feature_0_xyz = -480, -width_f0/2.0, -height_f0 / 2.0
feature_0 = plot_featuremap(feature_0_xyz, length_f0, width_f0, height_f0,
                           "368", "", "21")

start =20*height_f0/90.0
step = 25*height_f0/90.0
stop = start + step * 2 + 1
z_offsets = np.arange(start,stop,step) + feature_0_xyz[2]
for i_chan, z_offset in enumerate(z_offsets):
    ax.plot3D(np.linspace(feature_0_xyz[0],
                          feature_0_xyz[0]+length_f0,200), 
                          [feature_0_xyz[1]] * 200,
                          zs=9/2.0*cnt.data[12035:12235, i_chan] + z_offset,
                         color=seaborn.color_palette()[0])

filter_len = 150
filters_0_xyz = (feature_0_xyz[0] + length_f0/2.0 - filter_len/2.0, feature_0_xyz[1],
                 5*height_f0/90.0 + feature_0_xyz[2])
filters_0 = plot_filters(filters_0_xyz, 150, width_f0, 40,
                         "56", "", "1", boxpad=1, boxalpha=1)

# first featuremap
length_f1, width_f1, height_f1 = 420, 40 * 0.6, 90
feature_1_xyz = feature_0_xyz[0] + length_f0 + 300, -width_f1/2.0, -height_f1 /2.0
feature_1 = plot_featuremap(feature_1_xyz, length_f1,width_f1, height_f1, '313', '40', ' 21')
filters_1_xyz = (feature_1_xyz[0] + (length_f1)/2 -10,
              feature_1_xyz[1], feature_1_xyz[2]) 
filters_1 = plot_filters(filters_1_xyz, 0, width_f1, height_f1, '','40','21')



# square
square_x = feature_1_xyz[0] + length_f1

#second featuremap
length_f2, width_f2, height_f2 = 420, 40 * 0.6, one_height
feature_2_xyz = square_x + 400, -width_f2/2.0, -height_f2/2.0
plot_featuremap(feature_2_xyz, length_f2,width_f2, height_f2,
                  '105', '40', '')
pool_len = 200
filter_2_xyz = (feature_2_xyz[0] + (pool_len)/2 , feature_2_xyz[1], width_f2 * 0.4)
plot_filters(filter_2_xyz, pool_len, 7, height_f2,
                  '84', '1', '',)
# log
log_x = feature_2_xyz[0] + length_f2
# final softmax
length_soft, width_soft, height_soft = 100 * 2, 20 * 2 * 0.6, one_height
feature_soft_xyz = log_x + 250, -width_soft/2.0, -height_soft / 2.0
plot_filters(feature_soft_xyz,
            length_soft, width_soft, height_soft, '22', '40', '',topleft_annotate=False)

# final outs 
end_soft_x = feature_soft_xyz[0] + length_soft - width_soft * 0.8 # geraten :D
text_x = end_soft_x + 300

text_dist = 50

ax.text3D(text_x, 0, 0.5 * text_dist, zdir=(0,0,0),s='Normal', fontsize=10,va='center', ha='left')
ax.text3D(text_x, 0, -0.5 * text_dist, zdir=(0,0,0),s='Pathological', fontsize=10,va='center', ha='left')

# connector

middle_soft_z = feature_2_xyz[2] + width_soft * 0.6
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, 0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, -0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')



explain_fontsize=11
text_z_headers = 200 
ax.text3D(feature_0_xyz[0], 0, text_z_headers,
          'Convolution\n(temporal)', 
           weight='bold', ha='left',
         fontsize=explain_fontsize,  va='top',)
ax.text3D(feature_1_xyz[0] - 23, 0, text_z_headers,
          'Spatial filter\n(all electrodes,\nall previous filters)', 
          ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(feature_2_xyz[0] - 23, 0, text_z_headers,
          'Max Pooling\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(feature_soft_xyz[0] - 23, 0, text_z_headers,
          'Linear Classification\n(Dense Layer+Softmax)', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')

text_z_filt = text_z_headers - 75 
ax.text3D(feature_0_xyz[0], 0, text_z_filt,'24 Units\nStride 3x1', ha='left',
         fontsize=explain_fontsize,  va='top')
ax.text3D(feature_1_xyz[0] - 23, 0, text_z_filt,'73 Units', ha='left',
         fontsize=explain_fontsize,  va='top')
ax.text3D(feature_2_xyz[0] - 23, 0, text_z_filt, 
          'Stride 1x1', ha='left',
         fontsize=explain_fontsize,  va='top')
ax.text3D(feature_soft_xyz[0] - 23, 0, text_z_filt, '2 Units', ha='left',
         fontsize=explain_fontsize,  va='top')

ax.set_zlim(-1800,-800)
plt.savefig('/home/schirrmr/code/auto-diagnosis/ShallowSmacNet.svg', bbox_inches='tight', dpi=600, pad_inches=0)
plt.savefig('/home/schirrmr/code/auto-diagnosis/ShallowSmacNet.pdf', bbox_inches='tight', dpi=600, pad_inches=0)

In [ ]:
import braindevel
braindecode = braindevel
from braindevel.experiments.load import create_experiment
from braindevel.veganlasagne.layers import set_input_window_length, get_model_input_window
from braindevel.veganlasagne.layer_util import print_layers

exp = create_experiment('/home/schirrmr/data/models/paper/bci-competition/cnt/deep4/432.yaml',
                       change_braindecode_braindevel=True)
set_input_window_length(exp.final_layer, get_model_input_window(exp.final_layer))
print_layers(exp.final_layer)

In [ ]:
exp = create_experiment('/home/schirrmr/data/models/paper/bci-competition/epo/deep4/432.yaml')
set_input_window_length(exp.final_layer, get_model_input_window(exp.final_layer))
print_layers(exp.final_layer)

In [ ]:
from braindevel.experiments.load import create_experiment
from braindevel.veganlasagne.layers import set_input_window_length, get_model_input_window
from braindevel.veganlasagne.layer_util import print_layers

exp = create_experiment('/home/schirrmr/data/models/paper/bci-competition/cnt/shallow//332.yaml')
set_input_window_length(exp.final_layer, get_model_input_window(exp.final_layer))
print_layers(exp.final_layer)

In [ ]:
from braindevel.experiments.load import create_experiment
from braindevel.veganlasagne.layers import set_input_window_length, get_model_input_window
from braindevel.veganlasagne.layer_util import print_layers

exp = create_experiment('/home/schirrmr/data/models/paper/bci-competition/epo/shallow//332.yaml')
set_input_window_length(exp.final_layer, get_model_input_window(exp.final_layer))
print_layers(exp.final_layer)

In [ ]:
# probably deletable/unused this code
def elu(x):
    return (x >= 0) * x + (x < 0) * (np.exp(x) - 1)

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#%config InlineBackend.figure_format = 'png' 
#fig = plt.figure(figsize=(14,36))


%config InlineBackend.figure_format = 'svg' 
fig = plt.figure(figsize=(12,32))
ax = fig.gca(projection='3d')

ax.set_xlim3d(-185, 900)
ax.set_ylim3d(-100, 1300)
ax.set_zlim3d(-1400, 0)
ax.view_init(30,210)

#plt.tight_layout()
plt.axis('off')
no_color = (1,1,1,0)
black = (0,0,0,1)

featuremap_color = (1,1,1,0.0)
brown = seaborn.color_palette("BrBG")[0]#(132/255.0, 98/255.0, 98/255.0)
filter_color = seaborn.color_palette()[4] + (0.5,)#brown + (0.5,) #seaborn.color_palette()[4] + (0.7,)
activation_color = brown
text_filter_color = brown

filter_len = 120
pool_len = 80
one_height = 0

# rough width to height conversion
width_factor = 1.3 / 2.0


length_f0, width_f0, height_f0 = 2000, 0, 90
feature_0_xyz = -length_f0/2.0, -width_f0/2.0, 0
feature_0 = plot_featuremap(feature_0_xyz, length_f0, width_f0, height_f0,
                           "522", "", "21")

for i_chan, z_offset in enumerate((20,45,70)):
    ax.plot3D(np.linspace(feature_0_xyz[0],
                          feature_0_xyz[0]+length_f0,200), 
                          [feature_0_xyz[1]] * 200,
                          zs=9/2.0*cnt.data[12035:12235, i_chan] + z_offset,
                         color=seaborn.color_palette()[0])

    
filters_0_xyz = (-filter_len/2.0, feature_0_xyz[1], 2)
filters_0 = plot_filters(filters_0_xyz, filter_len, width_f0, 28,
                         "10", "", "1", boxpad=1, boxalpha=1)

# first featuremap
length_f1, width_f1, height_f1 = 1950, 50 * 0.6, 90
feature_1_xyz = -length_f1/2.0, -width_f1/2.0, -170
feature_1 = plot_featuremap(feature_1_xyz, length_f1,width_f1, height_f1, '513', '25', ' 44')
filters_1_xyz = (feature_1_xyz[0] + (length_f1)/2 -10,
              feature_1_xyz[1], feature_1_xyz[2]) 
filters_1 = plot_filters(filters_1_xyz, 0, width_f1, height_f1, '','25','44')

# square
elu_z = feature_1_xyz[2]
#elu_z = feature_1_xyz[2] - 75
#ax.text3D(0,0,elu_z, 'ELU', ha='center', fontsize=12, weight='bold',
#         backgroundcolor='white')


#second featuremap
length_f2, width_f2, height_f2 = 1950, 50 * 0.6, one_height
feature_2_xyz = -length_f2/2.0, -width_f2/2.0, elu_z-75
plot_featuremap(feature_2_xyz, length_f2,width_f2, height_f2,
                  '513', '25', '')
filter_2_xyz = (-pool_len / 2.0, 0, feature_2_xyz[2]+2)
plot_filters(filter_2_xyz, pool_len, 7, height_f2,
                  '3', '1', '',)
class ConvPoolBlock(object):
    def __init__(self, in_conv_length, in_conv_width, in_conv_height, in_conv_z,
                   filter_len, filt_width, filt_height, 
                   in_pool_len, in_pool_width, in_pool_conv_z_diff,
                   pool_len, pool_width, pool_height,
                   in_conv_len_str, in_conv_width_str, in_conv_height_str,
                   filt_len_str, filt_width_str, filt_height_str,
                   in_pool_len_str, in_pool_width_str, in_pool_height_str,
                   pool_len_str, pool_width_str, pool_height_str):
        self.__dict__.update(locals())
        del self.self
        self.in_conv_xyz = [-self.in_conv_length/2.0 - self.in_conv_width * width_factor / 2.0, 
                            -self.in_conv_width/2.0, 
                            self.in_conv_z]
        self.filt_xyz = [-self.filter_len / 2.0, -self.filt_width / 2.0 , 
            self.in_conv_z + self.in_conv_height / 2.0 - self.filt_height / 2.0]
        self.text_z = self.in_conv_xyz[2] - self.in_pool_conv_z_diff / 2.0
        self.in_pool_xyz = [-self.in_pool_len/2.0 - self.in_pool_width * width_factor /2.0, 
                            -self.in_pool_width/2.0, 
                            self.in_conv_z - self.in_pool_conv_z_diff]
        self.pool_xyz = [-self.pool_len / 2.0, -self.pool_width / 2.0, 
            self.in_pool_xyz[2] + self.in_conv_height / 2.0 - self.pool_height / 2.0]
        
    
    def draw(self):
        plot_featuremap(self.in_conv_xyz, self.in_conv_length, self.in_conv_width, self.in_conv_height,
                      self.in_conv_len_str, self.in_conv_width_str, self.in_conv_height_str)
        plot_filters(self.filt_xyz, self.filter_len, self.filt_width, self.filt_height,
                      self.filt_len_str, self.filt_width_str, self.filt_height_str,)
        
        #ax.text3D(0,0,self.text_z, 'ELU', ha='center', fontsize=12, weight='bold',
        # backgroundcolor='white')
        plot_featuremap(self.in_pool_xyz, self.in_pool_len, self.in_pool_width, self.in_conv_height,
                      self.in_pool_len_str, self.in_pool_width_str, self.in_pool_height_str)
        plot_filters(self.pool_xyz, self.pool_len, self.pool_width, self.pool_height,
                      self.pool_len_str, self.pool_width_str, self.pool_height_str)
        
        return self.in_conv_xyz, self.in_pool_xyz

length_f3, width_f3, height_f3 = 840, 50 * 0.6, one_height
conv_3_z = feature_2_xyz[2]-100
width_filt_3, height_filt_3 = width_f3, height_f3
pool_conv_z_diff = 90
in_pool_length_3, in_pool_width_3 = 780, 75 * 0.6
pool_width_3, pool_height_3 = 7, height_f3

block3 = ConvPoolBlock(
    length_f3, width_f3, height_f3,
    conv_3_z,
    filter_len,width_filt_3, height_filt_3, 
    in_pool_length_3, in_pool_width_3,
    pool_conv_z_diff,
    pool_len, pool_width_3, pool_height_3,
    '171', '25', '', 
    '10','25','',
    '162','50','',
    '3','1','')
block3.filt_xyz[0] += 20
conv_3_xyz, pool_3_xyz = block3.draw()

length, width, height = 420, 75 * 0.6, one_height
conv_z = pool_3_xyz[2] - 130
width_filt, height_filt = width, height
pool_conv_z_diff = 100
in_pool_length, in_pool_width = 400, 100 * 0.6
pool_width, pool_height = 7, height

block4 = ConvPoolBlock(
    length, width, height, 
    conv_z,
    filter_len,width_filt, height_filt, 
    in_pool_length, in_pool_width,
    pool_conv_z_diff,
    pool_len, pool_width, pool_height,
    '54', '50', '', 
    '10','50','',
    '45','100','',
    '3','1','')
conv_4_xyz, pool_4_xyz = block4.draw()

length, width, height = 320, 75 * 0.6, one_height
conv_z = pool_4_xyz[2] - 130
width_filt, height_filt = width, height
pool_conv_z_diff = 110
in_pool_length, in_pool_width = 200, 100 * 0.6
pool_width, pool_height = 7, height

block5 = ConvPoolBlock(
    length, width, height, 
    conv_z,
    filter_len,width_filt, height_filt, 
    in_pool_length, in_pool_width,
    pool_conv_z_diff,
    pool_len, pool_width, pool_height,
    '15', '100', '', 
    '10','100','',
    '6','200','',
    '3','1','')
block5.pool_xyz[0] = block5.in_pool_xyz[0] #tonio wanted left side
conv_5_xyz, pool_5_xyz = block5.draw()

# final softmax
length_soft, width_soft, height_soft = 100, 150 * 0.6, one_height
feature_soft_xyz = -length_soft/2.0, -width_soft/2.0, pool_5_xyz[2]-130
plot_filters(feature_soft_xyz,
            length_soft, width_soft, height_soft, '2', '200', '',
            topleft_annotate=False)

# final outs 

text_z = feature_soft_xyz[2] - 110

text_dist = 400

ax.text3D(-text_dist - text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Hand (L)', fontsize=12,va='top', ha='center')
ax.text3D(-text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Hand (R)', fontsize=12,va='top', ha='center')
ax.text3D(text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Feet', fontsize=12,va='top', ha='center')
ax.text3D(text_dist + text_dist / 2.0, 0, text_z, zdir=(0,0,0),s='Rest', fontsize=12,va='top', ha='center')

# connector
ax.plot3D([0, -text_dist - text_dist / 2.0], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([0,-text_dist / 2.0,], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([0,text_dist / 2.0], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([0,text_dist + text_dist / 2.0], ys=[feature_soft_xyz[1],0], zs=[feature_soft_xyz[2], text_z], linestyle='dotted',
         linewidth=1., color='black')
fig.savefig('to-delete.png')


explain_fontsize=12
text_x_headers = -1700 
ax.text3D(text_x_headers, 0, feature_0_xyz[2] + height_f0 * 1 + width_f0 * width_factor,
          'Convolution\n(temporal)', 
           weight='bold', ha='center',
         fontsize=explain_fontsize,  va='top',)
ax.text3D(text_x_headers, 0, feature_1_xyz[2] + height_f1 * 1 + width_f1 * width_factor,
          'Spatial filter\n(all electrodes,\nall previous filters)', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, feature_2_xyz[2] + height_f2 * 1 + width_f2 * width_factor,
          'Max Pooling\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, conv_3_xyz[2] + block3.in_conv_height + block3.in_conv_width * width_factor,
          'Convolution\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, pool_3_xyz[2] + block3.in_conv_height + block3.in_conv_width * width_factor,
          'Max Pooling\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, conv_4_xyz[2] + block4.in_conv_height + block4.in_conv_width * width_factor,
          'Convolution\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, pool_4_xyz[2] + block4.in_conv_height + block4.in_conv_width * width_factor,
          'Max Pooling\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, conv_5_xyz[2] + block5.in_conv_height + block5.in_conv_width * width_factor,
          'Convolution\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, pool_5_xyz[2] + block5.in_conv_height + block5.in_conv_width * width_factor,
          'Max Pooling\n', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_headers, 0, feature_soft_xyz[2] + height_soft * 1 + width_soft * width_factor,
          'Linear Classification\n(Dense Layer)', ha='center',
         weight='bold', fontsize=explain_fontsize,  va='top')

text_x_filt = text_x_headers 
offset_z = -52
ax.text3D(text_x_filt, 0, feature_0_xyz[2] + height_f0 * 1 + width_f0 * width_factor+offset_z,
          '25 Linear Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, feature_1_xyz[2] + height_f1 * 1 + width_f1 * width_factor+offset_z-20,
          '25 Exponential Linear Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, feature_2_xyz[2] + height_f2 * 1 + width_f2 * width_factor+offset_z/2.0, 
          'Stride 3x1', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, conv_3_xyz[2] +  block3.in_conv_height + block3.in_conv_width * width_factor +offset_z /2.0,
          '50 Exponential Linear Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, pool_3_xyz[2] + block3.in_conv_height + block3.in_conv_width * width_factor+offset_z/2.0, 
          'Stride 3x1', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, conv_4_xyz[2] +  block4.in_conv_height + block4.in_conv_width * width_factor +offset_z /2.0,
          '100 Exponential Linear  Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, pool_4_xyz[2] + block4.in_conv_height + block4.in_conv_width * width_factor+offset_z/2.0, 
          'Stride 3x1', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, conv_5_xyz[2] +  block5.in_conv_height + block5.in_conv_width * width_factor +offset_z /2.0,
          '200 Exponential Linear Units', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, pool_5_xyz[2] + block5.in_conv_height + block5.in_conv_width * width_factor+offset_z/2.0, 
          'Stride 3x1', ha='center',
         fontsize=explain_fontsize,  va='top')
ax.text3D(text_x_filt, 0, feature_soft_xyz[2] + height_soft * 1 + width_soft * width_factor+offset_z , 
          '4 Softmax Units', ha='center',
         fontsize=explain_fontsize,  va='top')


ax.set_zlim(-2600,-1200)
grey_blocks_width = 3400
grey_blocks_x_offset = -650

rect = add_rect(ax, (text_x_headers+grey_blocks_x_offset,
                     feature_0_xyz[2] + height_f0 * 1 + width_f0 * width_factor+30,),
                grey_blocks_width,-405,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (text_x_headers+grey_blocks_x_offset,
                     conv_3_xyz[2] + block3.in_conv_height + block3.in_conv_width * width_factor+30,)
                ,grey_blocks_width,-200,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (text_x_headers+grey_blocks_x_offset,
                     conv_4_xyz[2] + block4.in_conv_height + block4.in_conv_width * width_factor+30,)
                ,grey_blocks_width,-220,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (text_x_headers+grey_blocks_x_offset,
                     conv_5_xyz[2] + block5.in_conv_height + block5.in_conv_width * width_factor+30,)
                ,grey_blocks_width,-225,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (text_x_headers+grey_blocks_x_offset,
                     feature_soft_xyz[2] + height_soft * 1 + width_soft * width_factor+5,)
                ,grey_blocks_width,-225,
        edgecolor='black', facecolor=(0,0,0,0))

ax.text3D(text_x_headers+grey_blocks_x_offset+20, -3, feature_0_xyz[2] + height_f0 * 1 + width_f0 * width_factor+30 , 
          'Conv-Pool Block 1', ha='left', zdir=(-1e-6,0,-1),
         fontsize=explain_fontsize,  va='top',)
ax.text3D(text_x_headers+grey_blocks_x_offset+20, -3, conv_3_xyz[2] + block3.in_conv_height + block3.in_conv_width * width_factor+30 , 
          'Conv-Pool Block 2', ha='left', zdir=(-1e-6,0,-1),
         fontsize=explain_fontsize,  va='top',)
ax.text3D(text_x_headers+grey_blocks_x_offset+20, -3, conv_4_xyz[2] + block4.in_conv_height + block4.in_conv_width * width_factor+30 , 
          'Conv-Pool Block 3', ha='left', zdir=(-1e-6,0,-1),
         fontsize=explain_fontsize,  va='top',)
ax.text3D(text_x_headers+grey_blocks_x_offset+20, -3, conv_5_xyz[2] + block5.in_conv_height + block5.in_conv_width * width_factor+30 , 
          'Conv-Pool Block 4', ha='left', zdir=(-1e-6,0,-1),
         fontsize=explain_fontsize,  va='top',)
ax.text3D(text_x_headers+grey_blocks_x_offset+20, -3, feature_soft_xyz[2] + height_soft * 1 + width_soft * width_factor+5 , 
          'Classification Layer', ha='left', zdir=(-1e-6,0,-1),
         fontsize=explain_fontsize,  va='top',)
plt.savefig('3D_Diagram_MatplotLib.ipynb.1.pdf', bbox_inches='tight', dpi=600, pad_inches=0)

### plot horizontal, for slides

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#%config InlineBackend.figure_format = 'png' 
#fig = plt.figure(figsize=(14,36))


%config InlineBackend.figure_format = 'svg' 
fig = plt.figure(figsize=(14,30))
ax = fig.gca(projection='3d')

ax.set_xlim3d(-185, 1200)
ax.set_ylim3d(-100, 1300)
ax.set_zlim3d(-1400, 0)
ax.view_init(30,210)

#plt.tight_layout()
plt.axis('off')
no_color = (1,1,1,0)
black = (0,0,0,1)

featuremap_color = (1,1,1,0.0)
brown = seaborn.color_palette("BrBG")[0]#(132/255.0, 98/255.0, 98/255.0)
filter_color = seaborn.color_palette()[4] + (0.5,)#brown + (0.5,) #seaborn.color_palette()[4] + (0.7,)
activation_color = brown
text_filter_color = brown

filter_len = 40
pool_len = 70
one_height = 0
ax.set_zlim(-2600,-1200)#dlete again also

# rough width to height conversion
width_factor = 1.3 / 2.0


length_f0, width_f0, height_f0 = 380, 0, 120
feature_0_xyz = -3000, -width_f0/2.0, -height_f0/2.0
feature_0 = plot_featuremap(feature_0_xyz, length_f0, width_f0, height_f0,
                           "601", "", "21")

start =20*height_f0/90.0
step = 25*height_f0/90.0
stop = start + step * 2 + 1
z_offsets = np.arange(start,stop,step) + feature_0_xyz[2]
for i_chan, z_offset in enumerate(z_offsets):
    ax.plot3D(np.linspace(feature_0_xyz[0],
                          feature_0_xyz[0]+length_f0,200), 
                          [feature_0_xyz[1]] * 200,
                          zs=9/2.0*cnt.data[12035:12235, i_chan] + z_offset,
                         color=seaborn.color_palette()[0])    
filters_0_xyz = (feature_0_xyz[0] + length_f0/2.0 - filter_len/2.0, feature_0_xyz[1], 2*height_f0/90.0 + feature_0_xyz[2])
filters_0 = plot_filters(filters_0_xyz, filter_len, width_f0, 28 * height_f0/90.0,
                         "10", "", "1", boxpad=1, boxalpha=1)


# first featuremap
length_f1, width_f1, height_f1 = 330, 60 * 0.6, 100
feature_1_xyz = feature_0_xyz[0] + length_f0 + 300, -width_f1/2.0, -height_f1 /2.0
feature_1 = plot_featuremap(feature_1_xyz, length_f1,width_f1, height_f1, '592', '25', ' 21')
filters_1_xyz = (feature_1_xyz[0] + (length_f1)/2 -10,
              feature_1_xyz[1], feature_1_xyz[2]) 
filters_1 = plot_filters(filters_1_xyz, 0, width_f1, height_f1, '','25','21')

length_f2, width_f2, height_f2 = 330, 60 * 0.6, one_height
feature_2_xyz = feature_1_xyz[0] + length_f1 + 295, -width_f2/2.0, -height_f2 / 2.0
plot_featuremap(feature_2_xyz, length_f2,width_f2, height_f2,
                  '592', '25', '')
filter_2_xyz = (feature_2_xyz[0] + length_f2/2.0 - (pool_len)/2, 0, feature_2_xyz[2]+2)
plot_filters(filter_2_xyz, pool_len, 7, height_f2,
                  '3', '1', '',)



class ConvPoolBlock(object):
    def __init__(self, in_conv_length, in_conv_width, in_conv_height, in_conv_x,
                   filter_len, filt_width, filt_height, 
                   in_pool_len, in_pool_width, in_pool_conv_x_diff,
                   pool_len, pool_width, pool_height,
                   in_conv_len_str, in_conv_width_str, in_conv_height_str,
                   filt_len_str, filt_width_str, filt_height_str,
                   in_pool_len_str, in_pool_width_str, in_pool_height_str,
                   pool_len_str, pool_width_str, pool_height_str):
        self.__dict__.update(locals())
        del self.self
        self.in_conv_xyz = [in_conv_x, 
                            -self.in_conv_width/2.0, 
                            -self.in_conv_height/2.0]
        self.filt_xyz = [in_conv_x + self.in_conv_length / 2.0 -self.filter_len / 2.0,
                         -self.filt_width / 2.0 , 
            self.in_conv_xyz[2]]
        self.text_x = self.in_conv_xyz[0] + self.in_pool_conv_x_diff / 2.0 # unclear if used :))
        self.in_pool_xyz = [self.in_conv_x + self.in_pool_conv_x_diff, 
                            -self.in_pool_width/2.0, 
                            self.in_conv_xyz[2]]
        self.pool_xyz = [self.in_pool_xyz[0] + self.in_pool_len / 2.0 - self.pool_len / 2.0,
                         -self.pool_width / 2.0, 
                        self.in_pool_xyz[2]]
        
    
    def draw(self):
        plot_featuremap(self.in_conv_xyz, self.in_conv_length, self.in_conv_width, self.in_conv_height,
                      self.in_conv_len_str, self.in_conv_width_str, self.in_conv_height_str)
        plot_filters(self.filt_xyz, self.filter_len, self.filt_width, self.filt_height,
                      self.filt_len_str, self.filt_width_str, self.filt_height_str,)
        
        #ax.text3D(0,0,self.text_z, 'ELU', ha='center', fontsize=12, weight='bold',
        # backgroundcolor='white')
        plot_featuremap(self.in_pool_xyz, self.in_pool_len, self.in_pool_width, self.in_conv_height,
                      self.in_pool_len_str, self.in_pool_width_str, self.in_pool_height_str)
        plot_filters(self.pool_xyz, self.pool_len, self.pool_width, self.pool_height,
                      self.pool_len_str, self.pool_width_str, self.pool_height_str)
        
        return self.in_conv_xyz, self.in_pool_xyz

length_f3, width_f3, height_f3 = 180, 60 * 0.6, one_height
conv_3_x = feature_2_xyz[0] + length_f2 + 220
width_filt_3, height_filt_3 = width_f3, height_f3
pool_conv_x_diff = 380
in_pool_length_3, in_pool_width_3 = 150, 90 * 0.6
pool_width_3, pool_height_3 = 7, height_f3


block3 = ConvPoolBlock(
    length_f3, width_f3, height_f3,
    conv_3_x,
    filter_len,width_filt_3, height_filt_3, 
    in_pool_length_3, in_pool_width_3,
    pool_conv_x_diff,
    pool_len, pool_width_3, pool_height_3,
    '196', '25', '', 
    '10','25','',
    '187','50','',
    '3','1','')
#block3.filt_xyz[0] += 20
conv_3_xyz, pool_3_xyz = block3.draw()

length, width, height = 120, 90 * 0.6, one_height
conv_x = pool_3_xyz[0] + block3.pool_len + 350
width_filt, height_filt = width, height
pool_conv_x_diff = 380
in_pool_length, in_pool_width = 110, 120 * 0.6
pool_width, pool_height = 7, height

block4 = ConvPoolBlock(
    length, width, height, 
    conv_x,
    filter_len,width_filt, height_filt, 
    in_pool_length, in_pool_width,
    pool_conv_x_diff,
    pool_len, pool_width, pool_height,
    '61', '50', '', 
    '10','50','',
    '52','100','',
    '3','1','')
conv_4_xyz, pool_4_xyz = block4.draw()


length, width, height = 90, 120 * 0.6, one_height
conv_x = pool_4_xyz[0] + block4.pool_len + 360
width_filt, height_filt = width, height
pool_conv_x_diff = 385
in_pool_length, in_pool_width = 80, 150 * 0.6
pool_width, pool_height = 7, height

block5 = ConvPoolBlock(
    length, width, height, 
    conv_x,
    filter_len,width_filt, height_filt, 
    in_pool_length, in_pool_width,
    pool_conv_x_diff,
    pool_len, pool_width, pool_height,
    '16', '100', '', 
    '10','100','',
    '7','200','',
    '3','1','')
#block5.pool_xyz[0] = block5.in_pool_xyz[0] #tonio wanted left side
conv_5_xyz, pool_5_xyz = block5.draw()


## # final softmax
length_soft, width_soft, height_soft = 60, 150 * 0.6, one_height
feature_soft_xyz = pool_5_xyz[0] + block5.pool_len + 360, -width_soft/2.0, - height_soft /2.0
plot_filters(feature_soft_xyz,
            length_soft, width_soft, height_soft, '1', '200', '',
            topleft_annotate=False)

# final outs 
end_soft_x = feature_soft_xyz[0] + length_soft - width_soft * 0.8 # geraten :D
text_x = end_soft_x + 275

text_dist = 50

ax.text3D(text_x, 0, 0.5 * text_dist, zdir=(0,0,0),s='Normal', fontsize=10,va='center', ha='left')
ax.text3D(text_x, 0, -0.5 * text_dist, zdir=(0,0,0),s='Pathological', fontsize=10,va='center', ha='left')

# connector

middle_soft_z = feature_2_xyz[2] + width_soft * 0.6
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, 0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, -0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.set_zlim(-2600,-1200)#dlete again also

explain_fontsize=11
text_z_headers = 220 
ax.text3D(feature_0_xyz[0], 0, text_z_headers,
          'Convolution\n(temporal)', 
           weight='bold', ha='left',
         fontsize=explain_fontsize,  va='top',)
ax.text3D(feature_1_xyz[0] - width_f1*1, 0, text_z_headers,
          'Spatial filter\n(all electrodes,\nall prev. filters)', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(feature_2_xyz[0] - width_f2 - 50, 0, text_z_headers,
          'Max Pooling\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(conv_3_xyz[0] - block3.in_conv_width -70, 0, text_z_headers,
          'Conv\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(pool_3_xyz[0] - block3.in_pool_width -50, 0, text_z_headers,
          'Pool\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(conv_4_xyz[0] - block4.in_conv_width -50, 0, text_z_headers,
          'Conv\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(pool_4_xyz[0] - block4.in_pool_width -50, 0, text_z_headers,
          'Pool\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(conv_5_xyz[0] - block5.in_conv_width -70, 0, text_z_headers,
          'Conv\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(pool_5_xyz[0] - block5.in_pool_width -50, 0, text_z_headers,
          'Pool\n', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')
ax.text3D(feature_soft_xyz[0] - width_soft -50, 0, text_z_headers,
          'Linear Classification\n(Dense Layer)', ha='left',
         weight='bold', fontsize=explain_fontsize,  va='top')

explain_fontsize=10
offset_z = -75
ax.text3D(feature_0_xyz[0], 0, text_z_headers +offset_z,
          '25 Linear Units', 
           ha='left',
         fontsize=explain_fontsize,  va='top',)
ax.text3D(feature_1_xyz[0] - width_f1*1, 0, text_z_headers + offset_z,
          '25 Exponential\n Linear Units', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(feature_2_xyz[0] - width_f2 -50, 0, text_z_headers + offset_z,
          'Stride 3x1', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(conv_3_xyz[0] - block3.in_conv_width -70, 0, text_z_headers + offset_z,
          '50 ELUs\n', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(pool_3_xyz[0] - block3.in_pool_width - 50, 0, text_z_headers + offset_z,
          'Stride 3x1\n', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(conv_4_xyz[0] - block4.in_conv_width -50, 0, text_z_headers + offset_z,
          '100 ELUs\n', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(pool_4_xyz[0] - block4.in_pool_width -50, 0, text_z_headers + offset_z,
          'Stride 3x1\n', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(conv_5_xyz[0] - block5.in_conv_width -70, 0, text_z_headers + offset_z,
          '200 ELUs\n', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(pool_5_xyz[0] - block5.in_pool_width - 50, 0, text_z_headers + offset_z,
          'Stride 3x1\n', ha='left',
          fontsize=explain_fontsize,  va='top')
ax.text3D(feature_soft_xyz[0] - width_soft -50, 0, text_z_headers + offset_z,
          '2 Softmax Units', ha='left',
          fontsize=explain_fontsize,  va='top')


grey_blocks_height = 380
grey_blocks_z_offset = -650
grey_blocks_x_offset = -130
grey_blocks_z = -150
grey_blocks_x_pad = 20


rect = add_rect(ax, (feature_0_xyz[0]+grey_blocks_x_offset,
                     grey_blocks_z),
                conv_3_xyz[0] - feature_0_xyz[0] -10,grey_blocks_height,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (conv_3_xyz[0]+grey_blocks_x_offset + 5,
                     grey_blocks_z),
                conv_4_xyz[0] - conv_3_xyz[0] -10,grey_blocks_height,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (conv_4_xyz[0]+grey_blocks_x_offset + 5,
                     grey_blocks_z),
                conv_5_xyz[0] - conv_4_xyz[0] -50,grey_blocks_height,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (conv_5_xyz[0]+grey_blocks_x_offset -30,
                     grey_blocks_z),
                feature_soft_xyz[0] - conv_5_xyz[0] -25,grey_blocks_height,
        edgecolor='black', facecolor=(0,0,0,0))
rect = add_rect(ax, (feature_soft_xyz[0]+grey_blocks_x_offset -45,
                     grey_blocks_z),
                820,grey_blocks_height,
        edgecolor='black', facecolor=(0,0,0,0))

ax.text3D(feature_0_xyz[0]+grey_blocks_x_offset+20, 3, grey_blocks_z , 
          'Conv-Pool Block 1', ha='left', zdir=(1,0,0),
         fontsize=explain_fontsize,  va='bottom',)
ax.text3D(conv_3_xyz[0]+grey_blocks_x_offset + 5 +20, 3, grey_blocks_z , 
          'Conv-Pool Block 2', ha='left', zdir=(1,0,0),
         fontsize=explain_fontsize,  va='bottom',)
ax.text3D(conv_4_xyz[0]+grey_blocks_x_offset + 5 +20, 3, grey_blocks_z , 
          'Conv-Pool Block 3', ha='left', zdir=(1,0,0),
         fontsize=explain_fontsize,  va='bottom',)
ax.text3D(conv_5_xyz[0]+grey_blocks_x_offset -30 +20, 3, grey_blocks_z , 
          'Conv-Pool Block 4', ha='left', zdir=(1,0,0),
         fontsize=explain_fontsize,  va='bottom',)
ax.text3D(feature_soft_xyz[0]+grey_blocks_x_offset -45 +20, 3, grey_blocks_z , 
          'Classification Layer', ha='left', zdir=(1,0,0),
         fontsize=explain_fontsize,  va='bottom',)

plt.savefig('/home/schirrmr/code/auto-diagnosis/DeepNet.svg', bbox_inches='tight', dpi=600, pad_inches=0)
plt.savefig('/home/schirrmr/code/auto-diagnosis/DeepNet.pdf', bbox_inches='tight', dpi=600, pad_inches=0)

## For Press Release

In [ ]:
## import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#%config InlineBackend.figure_format = 'png' 
#fig = plt.figure(figsize=(14,36))


%config InlineBackend.figure_format = 'svg' 
fig = plt.figure(figsize=(14,30))
ax = fig.gca(projection='3d')

ax.set_xlim3d(-185, 1200)
ax.set_ylim3d(-100, 1300)
ax.set_zlim3d(-1400, 0)
ax.view_init(30,210)

#plt.tight_layout()
plt.axis('off')
no_color = (1,1,1,0)
black = (0,0,0,1)

featuremap_color = (1,1,1,0.0)
brown = seaborn.color_palette("BrBG")[0]#(132/255.0, 98/255.0, 98/255.0)
filter_color = seaborn.color_palette()[4] + (0.5,)#brown + (0.5,) #seaborn.color_palette()[4] + (0.7,)
activation_color = brown
text_filter_color = brown

filter_len = 40
pool_len = 70
one_height = 0
ax.set_zlim(-2600,-1200)#dlete again also

# rough width to height conversion
width_factor = 1.3 / 2.0


length_f0, width_f0, height_f0 = 380, 0, 120
feature_0_xyz = -3000, -width_f0/2.0, -height_f0/2.0
feature_0 = plot_featuremap(feature_0_xyz, length_f0, width_f0, height_f0,
                           "", "", "")

start =20*height_f0/90.0
step = 25*height_f0/90.0
stop = start + step * 2 + 1
z_offsets = np.arange(start,stop,step) + feature_0_xyz[2]
for i_chan, z_offset in enumerate(z_offsets):
    ax.plot3D(np.linspace(feature_0_xyz[0],
                          feature_0_xyz[0]+length_f0,200), 
                          [feature_0_xyz[1]] * 200,
                          zs=9/2.0*cnt.data[12035:12235, i_chan] + z_offset,
                         color=seaborn.color_palette()[0])    
filters_0_xyz = (feature_0_xyz[0] + length_f0/2.0 - filter_len/2.0, feature_0_xyz[1], 2*height_f0/90.0 + feature_0_xyz[2])
filters_0 = plot_filters(filters_0_xyz, filter_len, width_f0, 28 * height_f0/90.0,
                         "", "", "", boxpad=1, boxalpha=1)


# first featuremap
length_f1, width_f1, height_f1 = 330, 60 * 0.6, 100
feature_1_xyz = feature_0_xyz[0] + length_f0 + 150, -width_f1/2.0, -height_f1 /2.0
feature_1 = plot_featuremap(feature_1_xyz, length_f1,width_f1, height_f1, '', '', ' ')
filters_1_xyz = (feature_1_xyz[0] + (length_f1)/2 -10,
              feature_1_xyz[1], feature_1_xyz[2]) 
filters_1 = plot_filters(filters_1_xyz, 0, width_f1, height_f1, '','','')

length_f2, width_f2, height_f2 = 330, 60 * 0.6, one_height
feature_2_xyz = feature_1_xyz[0] + length_f1 + 150, -width_f2/2.0, -height_f2 / 2.0
plot_featuremap(feature_2_xyz, length_f2,width_f2, height_f2,
                  '', '', '')
filter_2_xyz = (feature_2_xyz[0] + length_f2/2.0 - (pool_len)/2, 0, feature_2_xyz[2]+2)
plot_filters(filter_2_xyz, pool_len, 7, height_f2,
                  '', '', '',)



class ConvPoolBlock(object):
    def __init__(self, in_conv_length, in_conv_width, in_conv_height, in_conv_x,
                   filter_len, filt_width, filt_height, 
                   in_pool_len, in_pool_width, in_pool_conv_x_diff,
                   pool_len, pool_width, pool_height,
                   in_conv_len_str, in_conv_width_str, in_conv_height_str,
                   filt_len_str, filt_width_str, filt_height_str,
                   in_pool_len_str, in_pool_width_str, in_pool_height_str,
                   pool_len_str, pool_width_str, pool_height_str):
        self.__dict__.update(locals())
        del self.self
        self.in_conv_xyz = [in_conv_x, 
                            -self.in_conv_width/2.0, 
                            -self.in_conv_height/2.0]
        self.filt_xyz = [in_conv_x + self.in_conv_length / 2.0 -self.filter_len / 2.0,
                         -self.filt_width / 2.0 , 
            self.in_conv_xyz[2]]
        self.text_x = self.in_conv_xyz[0] + self.in_pool_conv_x_diff / 2.0 # unclear if used :))
        self.in_pool_xyz = [self.in_conv_x + self.in_pool_conv_x_diff, 
                            -self.in_pool_width/2.0, 
                            self.in_conv_xyz[2]]
        self.pool_xyz = [self.in_pool_xyz[0] + self.in_pool_len / 2.0 - self.pool_len / 2.0,
                         -self.pool_width / 2.0, 
                        self.in_pool_xyz[2]]
        
    
    def draw(self):
        plot_featuremap(self.in_conv_xyz, self.in_conv_length, self.in_conv_width, self.in_conv_height,
                      self.in_conv_len_str, self.in_conv_width_str, self.in_conv_height_str)
        plot_filters(self.filt_xyz, self.filter_len, self.filt_width, self.filt_height,
                      self.filt_len_str, self.filt_width_str, self.filt_height_str,)
        
        #ax.text3D(0,0,self.text_z, 'ELU', ha='center', fontsize=12, weight='bold',
        # backgroundcolor='white')
        plot_featuremap(self.in_pool_xyz, self.in_pool_len, self.in_pool_width, self.in_conv_height,
                      self.in_pool_len_str, self.in_pool_width_str, self.in_pool_height_str)
        plot_filters(self.pool_xyz, self.pool_len, self.pool_width, self.pool_height,
                      self.pool_len_str, self.pool_width_str, self.pool_height_str)
        
        return self.in_conv_xyz, self.in_pool_xyz

length_f3, width_f3, height_f3 = 180, 60 * 0.6, one_height
conv_3_x = feature_2_xyz[0] + length_f2 + 100
width_filt_3, height_filt_3 = width_f3, height_f3
pool_conv_x_diff = 250
in_pool_length_3, in_pool_width_3 = 150, 90 * 0.6
pool_width_3, pool_height_3 = 7, height_f3


block3 = ConvPoolBlock(
    length_f3, width_f3, height_f3,
    conv_3_x,
    filter_len,width_filt_3, height_filt_3, 
    in_pool_length_3, in_pool_width_3,
    pool_conv_x_diff,
    pool_len, pool_width_3, pool_height_3,
    '', '', '', 
    '','','',
    '','','',
    '','','')
#block3.filt_xyz[0] += 20
conv_3_xyz, pool_3_xyz = block3.draw()

length, width, height = 120, 90 * 0.6, one_height
conv_x = pool_3_xyz[0] + block3.pool_len + 150
width_filt, height_filt = width, height
pool_conv_x_diff = 200
in_pool_length, in_pool_width = 110, 120 * 0.6
pool_width, pool_height = 7, height

block4 = ConvPoolBlock(
    length, width, height, 
    conv_x,
    filter_len,width_filt, height_filt, 
    in_pool_length, in_pool_width,
    pool_conv_x_diff,
    pool_len, pool_width, pool_height,
    '', '', '', 
    '','','',
    '','','',
    '','','')
conv_4_xyz, pool_4_xyz = block4.draw()


length, width, height = 90, 120 * 0.6, one_height
conv_x = pool_4_xyz[0] + block4.pool_len + 125
width_filt, height_filt = width, height
pool_conv_x_diff = 150
in_pool_length, in_pool_width = 80, 150 * 0.6
pool_width, pool_height = 7, height

block5 = ConvPoolBlock(
    length, width, height, 
    conv_x,
    filter_len,width_filt, height_filt, 
    in_pool_length, in_pool_width,
    pool_conv_x_diff,
    pool_len, pool_width, pool_height,
    '', '', '', 
    '','','',
    '','','',
    '','','')
#block5.pool_xyz[0] = block5.in_pool_xyz[0] #tonio wanted left side
conv_5_xyz, pool_5_xyz = block5.draw()


## # final softmax
length_soft, width_soft, height_soft = 60, 150 * 0.6, one_height
feature_soft_xyz = pool_5_xyz[0] + block5.pool_len + 100, -width_soft/2.0, - height_soft /2.0
plot_filters(feature_soft_xyz,
            length_soft, width_soft, height_soft, '', '', '',
            topleft_annotate=False)

# final outs 
end_soft_x = feature_soft_xyz[0] + length_soft - width_soft * 0.8 # geraten :D
text_x = end_soft_x + 250

text_dist = 50

ax.text3D(text_x, 0, 1.5 * text_dist, zdir=(0,0,0),s='Hand (L)', fontsize=16,va='center', ha='left')
ax.text3D(text_x, 0, 0.5 * text_dist, zdir=(0,0,0),s='Hand (R)', fontsize=16,va='center', ha='left')
ax.text3D(text_x, 0, -0.5 * text_dist, zdir=(0,0,0),s='Feet', fontsize=16,va='center', ha='left')
ax.text3D(text_x, 0, -1.5 * text_dist, zdir=(0,0,0),s='Rest', fontsize=16,va='center', ha='left')

# connector

middle_soft_z = feature_2_xyz[2] + width_soft * 0.6
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, 1.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, 0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, -0.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.plot3D([end_soft_x, text_x], ys=[feature_soft_xyz[1],0], zs=[middle_soft_z, -1.5*text_dist], linestyle='dotted',
         linewidth=1., color='black')
ax.set_zlim(-2600,-1200)#dlete again also

plt.savefig('Deep-convnet-press-release-larger-font.svg', bbox_inches='tight', dpi=600, pad_inches=0)

In [ ]:
from braindevel.experiments.load import load_exp_and_model
from braindevel.veganlasagne.layers import transform_to_normal_net

In [ ]:
exp, model =load_exp_and_model('data/models/paper/ours/cnt/shallow/2089')
print_layers(transform_to_normal_net(model))

In [ ]:
from braindevel.veganlasagne.layer_util import  print_layers

In [ ]:
exp, model =load_exp_and_model('data/models/paper/ours/cnt/deep4/4201')
print_layers(transform_to_normal_net(model))

In [ ]:
# somehow this plots correctly eventhough coordinates are very smal
# I don't understand meaning of text_bbox.x0's value
def text_data_bbox(text, renderer, ax):
    bbox = text.get_window_extent(renderer)
    transf = ax.transData.inverted()
    bb_datacoords = bbox.transformed(transf)
    return bb_datacoords

text_bbox = text_data_bbox(text_1, fig.canvas.renderer, ax)
artist = ax.add_artist(plt.Rectangle([text_bbox.x0, text_bbox.y0],
                            text_bbox.width * 1.2, text_bbox.height,
                           transform=ax.transData),
             )

In [ ]:
from braindevel.experiments.load import load_exp_and_model

In [ ]:
exp, model = load_exp_and_model('data/models/paper/ours/cnt/resnet/1')

In [ ]:
from braindevel.veganlasagne.layer_util import print_layers

In [ ]:
print_layers(model)

## Old stuff, other projections etc

#### Loading svg into cell to copy it

In [ ]:
fig.savefig('temp-to-delete.svg', format='svg')

In [ ]:
#%load temp-to-delete.svg

### Fixing axes code

In [ ]:
# fixing axes
from StringIO import StringIO


from xml.etree import ElementTree
string_file = StringIO()
fig.savefig(string_file, format='svg')
string_file.seek(0)
#print string_file.read()
#string_file.seek(0)
e = ElementTree.parse(string_file)

e = e.getroot()

for element in e.iter():
    if element.attrib.get('id', None) == 'line2d_1':
        axis_line = element
        print axis_line.getchildren()[0].attrib['d']

### Other projections code

In [ ]:
def cuboid_data(center, size):
    """
       Create a data array for cuboid plotting.


       ============= ================================================
       Argument      Description
       ============= ================================================
       center        center of the cuboid, triple
       size          size of the cuboid, triple, (x_length,y_width,z_height)
       :type size: tuple, numpy.array, list
       :param size: size of the cuboid, triple, (x_length,y_width,z_height)
       :type center: tuple, numpy.array, list
       :param center: center of the cuboid, triple, (x,y,z)


      """


    # suppose axis direction: x: to left; y: to inside; z: to upper
    # get the (left, outside, bottom) point
    o = [a - b / 2 for a, b in zip(center, size)]
    # get the length, width, and height
    l, w, h = size
    x = [[o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in bottom surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in upper surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],  # x coordinate of points in outside surface
         [o[0], o[0] + l, o[0] + l, o[0], o[0]]]  # x coordinate of points in inside surface
    y = [[o[1], o[1], o[1] + w, o[1] + w, o[1]],  # y coordinate of points in bottom surface
         [o[1], o[1], o[1] + w, o[1] + w, o[1]],  # y coordinate of points in upper surface
         [o[1], o[1], o[1], o[1], o[1]],          # y coordinate of points in outside surface
         [o[1] + w, o[1] + w, o[1] + w, o[1] + w, o[1] + w]]    # y coordinate of points in inside surface
    z = [[o[2], o[2], o[2], o[2], o[2]],                        # z coordinate of points in bottom surface
         [o[2] + h, o[2] + h, o[2] + h, o[2] + h, o[2] + h],    # z coordinate of points in upper surface
         [o[2], o[2], o[2] + h, o[2] + h, o[2]],                # z coordinate of points in outside surface
         [o[2], o[2], o[2] + h, o[2] + h, o[2]]]                # z coordinate of points in inside surface
    return x, y, z


import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
center = [0, 0, 0]
length = 15 * 2
width = 10 * 2
height = 100 * 2
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,5))
ax = fig.gca(projection='3d')
X, Y, Z = cuboid_data(center, (length, width, height))
ax.plot_surface(X, Y, Z, color='b', rstride=1, cstride=1, alpha=0.1)
ax.set_xlabel('X')
ax.set_xlim(-100, 100)
ax.set_ylabel('Y')
ax.set_ylim(-100, 100)
ax.set_zlabel('Z')
ax.set_zlim(-100, 100)
ax.view_init(elev=20, azim=20)
#plt.axis('off')


center = [50, 0, 0]
length = 15 * 2
width = 10 * 2
height = 100 * 2


X, Y, Z = cuboid_data(center, (length, width, height))
ax.plot_surface(X, Y, Z, color=seaborn.color_palette()[0],
                rstride=1, cstride=1, shade=False, alpha=0.3)
ax.set_xlim3d(-100, 500)

# fixing axes
from StringIO import StringIO


%config InlineBackend.figure_format = 'png' 
def orthogonal_proj(zfront, zback):
    a = (zfront+zback)/(zfront-zback)
    b = -2*(zfront*zback)/(zfront-zback)
    return np.array([[1,0,0,0],
                        [0.28867551,1,0,0],
                        [0,0,a,b],
                        [0,0,-1e-5,zback]])
proj3d.persp_transformation = orthogonal_proj
ax.view_init(30,300)
from xml.etree import ElementTree
string_file = StringIO()
fig.savefig(string_file, format='svg')
string_file.seek(0)
e = ElementTree.parse(string_file)

e = e.getroot()

for element in e.iter():
    if element.attrib.get('id', None) == 'line2d_1':
        axis_line = element
        print axis_line.getchildren()[0].attrib['d']
None